In [1]:
import numpy as np
import os
import pandas as pd
import copy
import matplotlib.pyplot as plt   
import seaborn as sns
import psycopg2
import pickle
from sklearn.svm import SVR
import warnings
warnings.filterwarnings("ignore")

In [2]:
connection = psycopg2.connect(database="edw_dev", user="postgres", password="AocAvalonPG123", host="52.15.157.95",port=5432)
cursor = connection.cursor()

# Iova Data

In [3]:
data1 = pd.read_sql_query('select edw_src.patient_info.gender,edw_fact.mme_details.master_procedure,edw_fact.mme_details.patient_type,edw_fact.mme_details.physician_name,edw_fact.mme_details.age,edw_fact.mme_details.bmi,edw_fact.mme_details.mme_medication_administrated_perdayavg from edw_fact.mme_details full outer join edw_src.patient_info on edw_fact.mme_details.unique_opd_id = edw_src.patient_info.procedure_id;',con=connection)
data1.head()

,gender,master_procedure,patient_type,physician_name,age,bmi,mme_medication_administrated_perdayavg
0,M,ARTHROSCOPY,OUT PATIENT,"WOLF, BRIAN R",41,36.44,12.00
1,F,ARTHROSCOPY SHOULDER,INPATIENT,"PATTERSON, BRENDAN M",69,37.51,45.00
2,M,ARTHROSCOPY,OUT PATIENT,"BOLLIER, MATTHEW J",63,38.02,30.34
3,M,ARTHROPLASTY,OUT PATIENT,"NOISEUX, NICOLAS O",65,31.68,53.83
4,M,ARTHROSCOPY SHOULDER,OUT PATIENT,"BOLLIER, MATTHEW J",61,24.91,30.00


# Oschner Data

In [4]:
data = pd.read_sql_query('select o.unique_case_id,o.patient_id ,o.order_id,o.administration_date ,o.gender ,o.age ,o.bmi ,o.cpt_code ,o.inpatient_or_outpatient ,o.procedure_name ,o.attending_physician ,o.drug_name ,o.drug_type ,p.master_procedure ,o.mme_medicationadministrated from edw_fact.dw_fact_oschner as o left join edw_dim.dw_dim_procedure as p on o.fk_procedure_id = p.procedure_id  ;',con=connection)
data.head()

,unique_case_id,patient_id,order_id,administration_date,gender,age,bmi,cpt_code,inpatient_or_outpatient,procedure_name,attending_physician,drug_name,drug_type,master_procedure,mme_medicationadministrated
0,857405,Z1828886,310196607,15-Nov-17,MALE,64,36.65,27447,Inpatient,"ARTHROPLASTY, KNEE",74917,HYDROMORPHONE MGML INJ UMBRELLA,INJ,"ARTHROPLASTY, KNEE",2.6
1,857405,Z1828886,310196607,15-Nov-17,MALE,64,36.65,27447,Inpatient,"ARTHROPLASTY, KNEE",74917,HYDROMORPHONE MGML INJ UMBRELLA,INJ,"ARTHROPLASTY, KNEE",2.6
2,857405,Z1828886,310244036,16-Nov-17,MALE,64,36.65,27447,Inpatient,"ARTHROPLASTY, KNEE",74917,OXYCODONE MG ORAL TAB,PO,"ARTHROPLASTY, KNEE",75.0
3,857405,Z1828886,310244036,15-Nov-17,MALE,64,36.65,27447,Inpatient,"ARTHROPLASTY, KNEE",74917,OXYCODONE MG ORAL TAB,PO,"ARTHROPLASTY, KNEE",75.0
4,857405,Z1828886,310244036,16-Nov-17,MALE,64,36.65,27447,Inpatient,"ARTHROPLASTY, KNEE",74917,OXYCODONE MG ORAL TAB,PO,"ARTHROPLASTY, KNEE",75.0


# Preprocess Data

In [5]:
def preprocess(data1,data2):
    data1 = data1.replace('INPATIENT','IN-PATIENT')
    data1 =  data1.dropna()
    data1 = data1.drop_duplicates()
    a = list(data1[data1['bmi'] > 50].index)
    data1 = data1.drop(a)
    
    data2 =  data2.dropna()
    data2 = data2.drop_duplicates()
    data2 = data2[data2['drug_type']=='PO']
    a = list(data2[data2['bmi'] > 50].index)
    data2 = data2.drop(a)
    b = data2[data2['mme_medicationadministrated']>300].index
    data2 = data2.drop(b)
    
    data2['patient_type'] = data2['inpatient_or_outpatient'].replace(['Inpatient','Outpatient',],['IN-PATIENT','OUT PATIENT'])
    df = data2[['gender','master_procedure','patient_type','attending_physician','age','bmi','mme_medicationadministrated']]
    df['gender'] = df['gender'].replace(['MALE','FEMALE',],['M','F'])
    df = df.rename(columns={"attending_physician": "physician_name","mme_medicationadministrated":"mme_medication_administrated_perdayavg"})
    df3 = pd.concat([data1, df], axis=0)
    return data1,df,df3
    

In [6]:
data1,df,df3 = preprocess(data1,data)
df3.shape

(46102, 7)

In [7]:
data1.shape[0] + df.shape[0]

46102

# Local Score

In [8]:
def local_score(df,doctor,patient_type):
    A = df[df['physician_name'] == doctor]
    B = A[A['patient_type'] == patient_type]
    B['Patient_category']= ['Old' if x>=45 else 'Adult' if 21<=x<45 else 'Young' for x in B['age']]
    B['BMI_category']= ['Under weight' if x<=18.5 else 'Normal' if 18.5<x<=24.5 else 'Over weight' for x in B['bmi']]
    C = B.groupby(['master_procedure','gender','Patient_category','BMI_category']).agg({"mme_medication_administrated_perdayavg": [ 'mean','count']})
    return C

# Globel Score

In [9]:
def globel_score(df,patient_type):
    B = df[df['patient_type'] == patient_type]
    B['Patient_category']= ['Old' if x>=45 else 'Adult' if 21<=x<45 else 'Young' for x in B['age']]
    B['BMI_category']= ['Under weight' if x<=18.5 else 'Normal' if 18.5<x<=24.5 else 'Over weight' for x in B['bmi']]
    C = B.groupby(['master_procedure','gender','Patient_category','BMI_category']).agg({"mme_medication_administrated_perdayavg": [ 'mean','count']})
    return C

# Doctor Score

In [10]:
def doctor_score(data,df3,doctor,patient_type):
    A = local_score(data,doctor,patient_type)
    proc = []
    gender = []
    pcate = []
    bcate = []
    for i in A.index:
        proc.append(i[0])
        gender.append(i[1])
        pcate.append(i[2])
        bcate.append(i[3])
    
    mean = []
    count  = []
    for i,j in zip(A[('mme_medication_administrated_perdayavg',  'mean')],A[('mme_medication_administrated_perdayavg', 'count')]):
        mean.append(i)
        count.append(j)
    
    S = pd.DataFrame({'master_procedure':proc,'gender':gender,'Patient_category':pcate,'BMI_category':bcate,'Local mean':mean,'Local count':count})
    
    B = globel_score(df3,patient_type)
    
    pproc = []
    ggender = []
    ppcate = []
    bbcate = []
    for i in B.index:
        pproc.append(i[0])
        ggender.append(i[1])
        ppcate.append(i[2])
        bbcate.append(i[3])
    
    meann = []
    countt  = []
    for i,j in zip(B[('mme_medication_administrated_perdayavg',  'mean')],B[('mme_medication_administrated_perdayavg', 'count')]):
        meann.append(i)
        countt.append(j)
    G = pd.DataFrame({'master_procedure':pproc,'gender':ggender,'Patient_category':ppcate,'BMI_category':bbcate,'Globel mean':meann,'Globel count':countt})

    D = pd.merge(S, G, on=['master_procedure','gender','Patient_category','BMI_category'], how='inner')
    
    Local_sum = {}
    globel_sum = {}
    for i in list(D['master_procedure'].unique()):
        A = D[D['master_procedure']==str(i)]
        LSUM,GSUM = A['Local count'].sum(),A['Globel count'].sum()
        Lsum = 0
        Gsum = 0
        for ind in A.index:
            a = A['Local count'][ind]
            b = A['Local mean'][ind]
            c = A['Globel count'][ind]
            d = A['Globel mean'][ind]
        
            Lsum += (a/LSUM)*b
            Gsum +=(c/GSUM)*d
        
        Local_sum[i] = Lsum
        globel_sum[i] = Gsum
        
    D['Local_Score'] = [Local_sum[i] for i in D['master_procedure']]
    D['Globel_Score'] = [globel_sum[i] for i in D['master_procedure']]
    
    D['Globel_relative_Score'] = [(j-i)/j for i,j in zip(D['Local_Score'],D['Globel_Score'])]
    D['Local_relative_Score'] = [(50-i)/50 for i in D['Local_Score']]
    
    print("Globel Score ",np.mean(list(D['Globel_relative_Score'].unique())))
    print("Local Score ",np.mean(list(D['Local_relative_Score'].unique())))
    
    return D
    

In [11]:
D = doctor_score(df,df3,'1887','OUT PATIENT')
D

Globel Score  0.12558177841710272
Local Score  -0.22777742760409175


,master_procedure,gender,Patient_category,BMI_category,Local mean,Local count,Globel mean,Globel count,Local_Score,Globel_Score,Globel_relative_Score,Local_relative_Score
0,"ARTHROPLASTY, KNEE",F,Adult,Over weight,150.000000,4,76.628649,116,115.271395,82.608258,-0.395398,-1.305428
1,"ARTHROPLASTY, KNEE",F,Old,Normal,131.578947,38,83.457573,423,115.271395,82.608258,-0.395398,-1.305428
2,"ARTHROPLASTY, KNEE",F,Old,Over weight,110.965074,408,83.426700,5018,115.271395,82.608258,-0.395398,-1.305428
3,"ARTHROPLASTY, KNEE",F,Old,Under weight,121.875000,4,115.178571,14,115.271395,82.608258,-0.395398,-1.305428
4,"ARTHROPLASTY, KNEE",M,Old,Normal,150.000000,6,69.671053,152,115.271395,82.608258,-0.395398,-1.305428
5,"ARTHROPLASTY, KNEE",M,Old,Over weight,119.736842,171,81.776912,2721,115.271395,82.608258,-0.395398,-1.305428
6,ARTHROSCOPY KNEE,F,Adult,Over weight,37.500000,2,56.610189,53,37.500000,51.084214,0.265918,0.250000
7,ARTHROSCOPY KNEE,M,Adult,Over weight,37.500000,1,48.472895,38,37.500000,51.084214,0.265918,0.250000
8,ARTHROSCOPY KNEE,M,Young,Over weight,37.500000,3,47.132245,49,37.500000,51.084214,0.265918,0.250000
9,Anterior Cruciate Ligament Reconstruction,F,Adult,Normal,37.500000,3,63.196721,61,52.159091,74.065041,0.295766,-0.043182


In [12]:
E = doctor_score(data1,df3,'WOLF, BRIAN R','OUT PATIENT')
E

Globel Score  -0.06013237393773384
Local Score  -0.08780846296063739


,master_procedure,gender,Patient_category,BMI_category,Local mean,Local count,Globel mean,Globel count,Local_Score,Globel_Score,Globel_relative_Score,Local_relative_Score
0,ANT CRUC LIGAMENT,F,Adult,Normal,49.037500,8,51.352000,20,50.626154,46.815088,-0.081407,-0.012523
1,ANT CRUC LIGAMENT,F,Adult,Over weight,51.171111,9,47.176552,58,50.626154,46.815088,-0.081407,-0.012523
2,ANT CRUC LIGAMENT,F,Old,Normal,30.333333,3,35.166667,6,50.626154,46.815088,-0.081407,-0.012523
3,ANT CRUC LIGAMENT,F,Old,Over weight,55.660000,2,42.434118,17,50.626154,46.815088,-0.081407,-0.012523
4,ANT CRUC LIGAMENT,F,Young,Normal,42.311429,7,36.567692,26,50.626154,46.815088,-0.081407,-0.012523
...,...,...,...,...,...,...,...,...,...,...,...,...
115,VS,F,Young,Over weight,40.000000,1,40.000000,1,54.013333,54.145417,0.002439,-0.080267
116,VS,M,Adult,Normal,57.560000,3,54.420000,4,54.013333,54.145417,0.002439,-0.080267
117,VS,M,Adult,Over weight,45.000000,1,55.604000,5,54.013333,54.145417,0.002439,-0.080267
118,VS,M,Young,Normal,40.320000,2,49.128000,5,54.013333,54.145417,0.002439,-0.080267


In [13]:
E = doctor_score(data1,df3,'BOLLIER, MATTHEW J','OUT PATIENT')
E

Globel Score  0.12137533026474126
Local Score  0.13644489620010478


,master_procedure,gender,Patient_category,BMI_category,Local mean,Local count,Globel mean,Globel count,Local_Score,Globel_Score,Globel_relative_Score,Local_relative_Score
0,ANT CRUC LIGAMENT,F,Adult,Normal,55.420000,6,51.352000,20,43.571260,46.802010,0.069030,0.128575
1,ANT CRUC LIGAMENT,F,Adult,Over weight,44.410000,21,47.176552,58,43.571260,46.802010,0.069030,0.128575
2,ANT CRUC LIGAMENT,F,Old,Normal,40.000000,3,35.166667,6,43.571260,46.802010,0.069030,0.128575
3,ANT CRUC LIGAMENT,F,Old,Over weight,39.980000,14,42.434118,17,43.571260,46.802010,0.069030,0.128575
4,ANT CRUC LIGAMENT,F,Young,Normal,36.387273,11,36.567692,26,43.571260,46.802010,0.069030,0.128575
...,...,...,...,...,...,...,...,...,...,...,...,...
149,POSTERIOR CRUCIATE LIGAMENT (PCL) RECONSTRUCTI...,M,Young,Over weight,73.750000,2,61.393333,3,64.304211,64.059216,-0.003825,-0.286084
150,Rotator Cuff Repair,F,Old,Normal,30.000000,2,85.227273,11,36.220952,68.642295,0.472323,0.275581
151,Rotator Cuff Repair,F,Old,Over weight,37.500000,2,66.295455,55,36.220952,68.642295,0.472323,0.275581
152,Rotator Cuff Repair,M,Old,Over weight,39.515556,3,67.901204,72,36.220952,68.642295,0.472323,0.275581


In [14]:
E.shape

(154, 12)